# Direct stiffness method

> A custom solver for trusses

https://en.wikipedia.org/wiki/Direct_stiffness_method

In [ ]:
# | default_exp _direct_stiffness_method

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

In [ ]:
from rich import print

In [ ]:
# | export
from fastcore.basics import patch
import numpy as np
import pandas as pd

Each `DirectStiffnessMethod` does calculate for one structural analysis model's load group the results.

In [ ]:
# | export
class DirectStiffnessMethodException(Exception):
    pass

In [ ]:
# | export
class DirectStiffnessMethod:
    def __init__(
        self,
        *,
        bars,
        nodes,
        point_loads,
    ):
        self.bars = bars
        self.nodes = nodes
        self.point_loads = point_loads

In [ ]:
Nodes_data = {
    "Node": pd.Series([2, 1, 3, 4], dtype=int),
    "Coordinate_X": pd.Series([0, 0, -4e3, -4e3], dtype=float),
    "Coordinate_Y": pd.Series([0, 0, 0, 0], dtype=float),
    "Coordinate_Z": pd.Series([3e3, 0, 3e3, 6e3], dtype=float),
    "Translational_X": pd.Series([0, 1, 1, 1], dtype=bool),
    "Translational_Y": pd.Series([1, 1, 1, 1], dtype=bool),
    "Translational_Z": pd.Series([0, 1, 1, 1], dtype=bool),
}

Nodes = pd.DataFrame(Nodes_data)
Nodes.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z
2,0.000000,0.000000,3000.000000,False,True,False
1,0.000000,0.000000,0.000000,True,True,True
3,-4000.000000,0.000000,3000.000000,True,True,True
4,-4000.000000,0.000000,6000.000000,True,True,True


In [ ]:
Bars_data = {
    "Bar": pd.Series([1, 2, 3], dtype=int),
    "Start_node": pd.Series([2, 2, 2], dtype=int),
    "End_node": pd.Series([1, 3, 4], dtype=int),
    "Cross-sectional_area": pd.Series([1e3, 1e3, 1e3], dtype=float),
    "Modulus_of_elasticity": pd.Series([1e3, 1e3, 1e3], dtype=float),
}
Bars = pd.DataFrame(Bars_data)
Bars.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity
1,2,1,1000.000000,1000.000000
2,2,3,1000.000000,1000.000000
3,2,4,1000.000000,1000.000000


In [ ]:
Point_Loads_data = {
    "Point_Load": pd.Series(
        [
            1,
            2,
            3,
        ],
        dtype=int,
    ),
    "Node": pd.Series(
        [2, 4, 4],
        dtype=int,
    ),
    "Force_X": pd.Series(
        [100e3, 100e3, 50e3],
        dtype=float,
    ),
    "Force_Y": pd.Series(
        [0, 0, 0],
        dtype=float,
    ),
    "Force_Z": pd.Series(
        [-100e3, 0, 50e3],
        dtype=float,
    ),
}
Point_Loads = pd.DataFrame(Point_Loads_data)
Point_Loads.style.hide(axis="index")

Point_Load,Node,Force_X,Force_Y,Force_Z
1,2,100000.000000,0.000000,-100000.000000
2,4,100000.000000,0.000000,0.000000
3,4,50000.000000,0.000000,50000.000000


In [ ]:
System = DirectStiffnessMethod(
    bars=Bars,
    nodes=Nodes,
    point_loads=Point_Loads,
)

In [ ]:
System.nodes.shape[0]

4

In [ ]:
# | export
@patch
def extend_nodes_df(
    self: DirectStiffnessMethod,
):
    self.number_of_rows = self.nodes.shape[0]
    self.dimensions = 3

    self.number_of_degrees_of_freedom = (
        self.number_of_rows * self.dimensions
    )

    self.nodes["Degrees_of_freedom"] = pd.Series(
        np.split(
            np.arange(self.number_of_degrees_of_freedom),
            self.number_of_rows,
        )
    )

In [ ]:
System.extend_nodes_df()

In [ ]:
System.nodes.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z,Degrees_of_freedom
2,0.000000,0.000000,3000.000000,False,True,False,[0 1 2]
1,0.000000,0.000000,0.000000,True,True,True,[3 4 5]
3,-4000.000000,0.000000,3000.000000,True,True,True,[6 7 8]
4,-4000.000000,0.000000,6000.000000,True,True,True,[ 9 10 11]


In [ ]:
# | export
@patch
def extend_bars_df(
    self: DirectStiffnessMethod,
):
    start_node = self.nodes.rename(
        columns={
            "Node": "Start_node",
            "Coordinate_X": "x_1",
            "Coordinate_Y": "y_1",
            "Coordinate_Z": "z_1",
            "Degrees_of_freedom": "Degrees_of_freedom_1",
        }
    )

    bars_start = pd.merge(
        self.bars,
        start_node[
            ["Start_node", "x_1", "y_1", "z_1", "Degrees_of_freedom_1"]
        ],
        on="Start_node",
    )

    end_node = self.nodes.rename(
        columns={
            "Node": "End_node",
            "Coordinate_X": "x_2",
            "Coordinate_Y": "y_2",
            "Coordinate_Z": "z_2",
            "Degrees_of_freedom": "Degrees_of_freedom_2",
        }
    )

    self.bars_extended = pd.merge(
        bars_start,
        end_node[["End_node", "x_2", "y_2", "z_2", "Degrees_of_freedom_2"]],
        on="End_node",
    )

In [ ]:
System.extend_bars_df()

In [ ]:
System.bars_extended.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity,x_1,y_1,z_1,Degrees_of_freedom_1,x_2,y_2,z_2,Degrees_of_freedom_2
1,2,1,1000.000000,1000.000000,0.000000,0.000000,3000.000000,[0 1 2],0.000000,0.000000,0.000000,[3 4 5]
2,2,3,1000.000000,1000.000000,0.000000,0.000000,3000.000000,[0 1 2],-4000.000000,0.000000,3000.000000,[6 7 8]
3,2,4,1000.000000,1000.000000,0.000000,0.000000,3000.000000,[0 1 2],-4000.000000,0.000000,6000.000000,[ 9 10 11]


$$l_x = x_2 - x_1, \quad l_y = y_2 - y_1, \quad l_z = z_2 - z_1$$

In [ ]:
# | export
@patch
def calculate_differences(
    self: DirectStiffnessMethod, x_1, y_1, z_1, x_2, y_2, z_2
):
    l_x = x_2 - x_1
    l_y = y_2 - y_1
    l_z = z_2 - z_1
    return l_x, l_y, l_z

$$l = \sqrt{l_x^2 + l_y^2 + l_z^2}$$

In [ ]:
# | export
@patch
def calculate_distance(self: DirectStiffnessMethod, l_x, l_y, l_z):
    vector = np.array([l_x, l_y, l_z])
    l = np.linalg.norm(vector)
    return l

$$K_e = \frac{E \cdot A}{l^3} \cdot \begin{bmatrix}
l_x^2 & l_x \cdot l_y & l_x \cdot l_z & -l_x^2 & -l_x \cdot l_y & -l_x \cdot l_z \\
l_x \cdot l_y & l_y^2 & l_y \cdot l_z & -l_x \cdot l_y & -l_y^2 & -l_y \cdot l_z \\
l_x \cdot l_z & l_y \cdot l_z & l_z^2 & -l_x \cdot l_z & -l_y \cdot l_z & -l_z^2 \\
-l_x^2 & -l_x \cdot l_y & -l_x \cdot l_z & l_x^2 & l_x \cdot l_y & l_x \cdot l_z \\
-l_x \cdot l_y & -l_y^2 & -l_y \cdot l_z & l_x \cdot l_y & l_y^2 & l_y \cdot l_z \\
-l_x \cdot l_z & -l_y \cdot l_z & -l_z^2 & l_x \cdot l_z & l_y \cdot l_z & l_z^2 \\
\end{bmatrix}$$

In [ ]:
# | export
@patch
def create_element_stiffness_matrice(
    self: DirectStiffnessMethod, E, A, x_1, y_1, z_1, x_2, y_2, z_2
):
    l_x, l_y, l_z = self.calculate_differences(x_1, y_1, z_1, x_2, y_2, z_2)

    l = self.calculate_distance(l_x, l_y, l_z)

    K_e = (E * A / (l**3)) * np.array(
        [
            [
                l_x**2,
                l_x * l_y,
                l_x * l_z,
                -(l_x**2),
                -l_x * l_y,
                -l_x * l_z,
            ],
            [
                l_x * l_y,
                l_y**2,
                l_y * l_z,
                -l_x * l_y,
                -(l_y**2),
                -l_y * l_z,
            ],
            [
                l_x * l_z,
                l_y * l_z,
                l_z**2,
                -l_x * l_z,
                -l_y * l_z,
                -(l_z**2),
            ],
            [
                -(l_x**2),
                -l_x * l_y,
                -l_x * l_z,
                l_x**2,
                l_x * l_y,
                l_x * l_z,
            ],
            [
                -l_x * l_y,
                -(l_y**2),
                -l_y * l_z,
                l_x * l_y,
                l_y**2,
                l_y * l_z,
            ],
            [
                -l_x * l_z,
                -l_y * l_z,
                -(l_z**2),
                l_x * l_z,
                l_y * l_z,
                l_z**2,
            ],
        ]
    )

    return K_e

In [ ]:
# | export
@patch
def create_element_stiffness_matrices(
    self: DirectStiffnessMethod,
):
    self.element_stiffness_matrices = [
        (
            name,
            self.create_element_stiffness_matrice(
                E, A, x_1, y_1, z_1, x_2, y_2, z_2
            ),
            np.concatenate((degrees_of_freedom_1, degrees_of_freedom_2)),
        )
        for name, E, A, x_1, y_1, z_1, x_2, y_2, z_2, degrees_of_freedom_1, degrees_of_freedom_2 in zip(
            self.bars_extended["Bar"],
            self.bars_extended["Modulus_of_elasticity"],
            self.bars_extended["Cross-sectional_area"],
            self.bars_extended["x_1"],
            self.bars_extended["y_1"],
            self.bars_extended["z_1"],
            self.bars_extended["x_2"],
            self.bars_extended["y_2"],
            self.bars_extended["z_2"],
            self.bars_extended["Degrees_of_freedom_1"],
            self.bars_extended["Degrees_of_freedom_2"],
        )
    ]

    self.element_stiffness_matrices_df = pd.DataFrame(
        self.element_stiffness_matrices,
        columns=["Bar", "Element_Matrice", "Degrees_of_freedom"],
    )

In [ ]:
System.create_element_stiffness_matrices()

In [ ]:
System.element_stiffness_matrices_df

,Bar,Element_Matrice,Degrees_of_freedom
0,1,"[[0.0, 0.0, -0.0, -0.0, -0.0, 0.0], [0.0, 0.0,...","[0, 1, 2, 3, 4, 5]"
1,2,"[[250.0, -0.0, -0.0, -250.0, 0.0, 0.0], [-0.0,...","[0, 1, 2, 6, 7, 8]"
2,3,"[[128.0, -0.0, -96.0, -128.0, 0.0, 96.0], [-0....","[0, 1, 2, 9, 10, 11]"


In [ ]:
print(System.element_stiffness_matrices[2][1])

[[ 128.   -0.  -96. -128.    0.   96.]
 [  -0.    0.    0.    0.   -0.   -0.]
 [ -96.    0.   72.   96.   -0.  -72.]
 [-128.    0.   96.  128.   -0.  -96.]
 [   0.   -0.   -0.   -0.    0.    0.]
 [  96.   -0.  -72.  -96.    0.   72.]]

In [ ]:
test = np.array(
    [
        [
            1,
            2,
            30,
        ],
        [
            6,
            4,
            80,
        ],
        [
            5,
            8,
            70,
        ],
    ]
)

In [ ]:
index = np.array([1, 5, 6])

In [ ]:
original_array = np.array([1, 5, 6])
replicated_array = np.tile(original_array, 3)
print(replicated_array)

[1 5 6 1 5 6 1 5 6]

In [ ]:
original_array = np.array([1, 5, 6])
repeated_array = np.repeat(
    original_array, 3
)  # Repeats each element 3 times
print(repeated_array)

[1 1 1 5 5 5 6 6 6]

In [ ]:
for idx_horizontal, idx_vertical, value in zip(
    np.repeat(original_array, 3), np.tile(original_array, 3), test.flat
):
    print(idx_horizontal, idx_vertical, value)

1 1 1

1 5 2

1 6 30

5 1 6

5 5 4

5 6 80

6 1 5

6 5 8

6 6 70

In [ ]:
# | export
@patch
def create_system_stiffness_matrice(
    self: DirectStiffnessMethod,
):
    self.K_f = np.zeros(
        (
            self.number_of_degrees_of_freedom,
            self.number_of_degrees_of_freedom,
        )
    )

    for element_matrice, degrees_of_freedom in zip(
        self.element_stiffness_matrices_df["Element_Matrice"],
        self.element_stiffness_matrices_df["Degrees_of_freedom"],
    ):
        for idx_horizontal, idx_vertical, value in zip(
            np.repeat(degrees_of_freedom, len(degrees_of_freedom)),
            np.tile(degrees_of_freedom, len(degrees_of_freedom)),
            element_matrice.flat,
        ):
            self.K_f[idx_horizontal, idx_vertical] += value

In [ ]:
System.create_system_stiffness_matrice()

In [ ]:
print(System.K_f)

[[ 378.            0.          -96.            0.            0.
     0.         -250.            0.            0.         -128.
     0.           96.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [ -96.            0.          405.33333333    0.            0.
  -333.33333333    0.            0.            0.           96.
     0.          -72.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.         -333.33333333    0.            0.
   333.33333333    0.            0.            0.            0.
     0.            0.        ]
 [-250.            0.            0.            0.            0.
     0.          250.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [-128.            0.           96.            0.            0.
     0.            0.            0.            0.          128.
     0.          -96.        ]
 [   0.            0.            0.            0.            0.
     0.            0.            0.            0.            0.
     0.            0.        ]
 [  96.            0.          -72.            0.            0.
     0.            0.            0.            0.          -96.
     0.           72.        ]]

In [ ]:
print(System.K_f.__repr__())

array([[ 378.        ,    0.        ,  -96.        ,    0.        ,
           0.        ,    0.        , -250.        ,    0.        ,
           0.        , -128.        ,    0.        ,   96.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [ -96.        ,    0.        ,  405.33333333,    0.        ,
           0.        , -333.33333333,    0.        ,    0.        ,
           0.        ,   96.        ,    0.        ,  -72.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        , -333.33333333,    0.        ,
           0.        ,  333.33333333,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [-250.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,  250.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [-128.        ,    0.        ,   96.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,  128.        ,    0.        ,  -96.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [  96.        ,    0.        ,  -72.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,  -96.        ,    0.        ,   72.        ]])

In [ ]:
System.nodes

,Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z,Degrees_of_freedom
0,2,0.0,0.0,3000.0,False,True,False,"[0, 1, 2]"
1,1,0.0,0.0,0.0,True,True,True,"[3, 4, 5]"
2,3,-4000.0,0.0,3000.0,True,True,True,"[6, 7, 8]"
3,4,-4000.0,0.0,6000.0,True,True,True,"[9, 10, 11]"


In [ ]:
# | export
@patch
def create_force_vector(
    self: DirectStiffnessMethod,
):
    self.F_f = np.zeros(self.number_of_degrees_of_freedom)

    self.point_loads_extended = pd.merge(
        self.point_loads,
        self.nodes[["Node", "Degrees_of_freedom"]],
        on="Node",
    )

    np.add.at(
        self.F_f,
        np.concatenate(
            self.point_loads_extended["Degrees_of_freedom"].values
        ),
        np.concatenate(
            self.point_loads_extended[
                ["Force_X", "Force_Y", "Force_Z"]
            ].values
        ),
    )

In [ ]:
System.create_force_vector()

In [ ]:
System.F_f

array([ 100000.,       0., -100000.,       0.,       0.,       0.,
             0.,       0.,       0.,  150000.,       0.,   50000.])

In [ ]:
X_d = np.concatenate(
    System.point_loads_extended["Degrees_of_freedom"].values
)

In [ ]:
X = np.concatenate(
    System.point_loads_extended[["Force_X", "Force_Y", "Force_Z"]].values
)

In [ ]:
F_f = np.zeros(System.number_of_rows * System.dimensions)

In [ ]:
F_f

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
np.add.at(F_f, X_d, X)

In [ ]:
F_f

array([ 100000.,       0., -100000.,       0.,       0.,       0.,
             0.,       0.,       0.,  150000.,       0.,   50000.])

In [ ]:
System.F_f

array([ 100000.,       0., -100000.,       0.,       0.,       0.,
             0.,       0.,       0.,  150000.,       0.,   50000.])

In [ ]:
System.point_loads_extended

,Point_Load,Node,Force_X,Force_Y,Force_Z,Degrees_of_freedom
0,1,2,100000.0,0.0,-100000.0,"[0, 1, 2]"
1,2,4,100000.0,0.0,0.0,"[9, 10, 11]"
2,3,4,50000.0,0.0,50000.0,"[9, 10, 11]"


In [ ]:
translationals = [True, False, True]

In [ ]:
test

array([[ 1,  2, 30],
       [ 6,  4, 80],
       [ 5,  8, 70]])

In [ ]:
test[:, translationals][translationals]

array([[ 1, 30],
       [ 5, 70]])

In [ ]:
System.nodes

,Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z,Degrees_of_freedom
0,2,0.0,0.0,3000.0,False,True,False,"[0, 1, 2]"
1,1,0.0,0.0,0.0,True,True,True,"[3, 4, 5]"
2,3,-4000.0,0.0,3000.0,True,True,True,"[6, 7, 8]"
3,4,-4000.0,0.0,6000.0,True,True,True,"[9, 10, 11]"


In [ ]:
System.nodes[
    [
        "Translational_X",
        "Translational_Y",
        "Translational_Z",
    ]
].values

array([[False,  True, False],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [ ]:
np.logical_not(
    System.nodes[
        [
            "Translational_X",
            "Translational_Y",
            "Translational_Z",
        ]
    ].values.flatten()
)

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False])

In [ ]:
# | export
@patch
def create_reduced_system_stiffness_matrice_and_reduced_force_vector(
    self: DirectStiffnessMethod,
):
    self.translationals = np.logical_not(
        self.nodes[
            [
                "Translational_X",
                "Translational_Y",
                "Translational_Z",
            ]
        ].values.flatten()
    )
    self.K_f_reduced = self.K_f[:, self.translationals][self.translationals]
    self.F_f_reduced = self.F_f[self.translationals]

In [ ]:
System.create_reduced_system_stiffness_matrice_and_reduced_force_vector()

In [ ]:
System.translationals

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False])

In [ ]:
System.F_f_reduced

array([ 100000., -100000.])

In [ ]:
System.K_f_reduced

array([[378.        , -96.        ],
       [-96.        , 405.33333333]])

$$K_{f,reduced}^{-1} \cdot F_{f,reduced} = u_{f,reduced}$$

In [ ]:
# | export
@patch
def calculate_displacement_vector(
    self: DirectStiffnessMethod,
):
    try:
        self.u_f_reduced = (
            np.linalg.inv(self.K_f_reduced) @ self.F_f_reduced
        )
    except np.linalg.LinAlgError as l:
        if str(l) == "Singular matrix":
            raise DirectStiffnessMethodException(
                "Mechanism - The system is unstable"
            )
        else:
            raise l

    self.degrees_of_freedom = np.concatenate(
        self.nodes[
            [
                "Degrees_of_freedom",
            ]
        ].values.flatten()
    )
    self.u_f = np.zeros(
        self.number_of_rows * self.dimensions,
    )

    self.u_f[self.translationals] = self.u_f_reduced

In [ ]:
System.calculate_displacement_vector()

In [ ]:
System.u_f_reduced

array([ 214.81481481, -195.83333333])

In [ ]:
System.u_f

array([ 214.81481481,    0.        , -195.83333333,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

$$K_{f} \cdot u_{f} = F_{f}$$

In [ ]:
# | export
@patch
def calculate_force_vector(
    self: DirectStiffnessMethod,
):
    self.F_f_calculated = self.K_f @ self.u_f

In [ ]:
System.calculate_force_vector()

In [ ]:
System.F_f_calculated

array([ 100000.        ,       0.        , -100000.        ,
             0.        ,       0.        ,   65277.77777778,
        -53703.7037037 ,       0.        ,       0.        ,
        -46296.2962963 ,       0.        ,   34722.22222222])

If the user has defined a point load in the support, this force must be considered in the support forces. This is achieved with the function `correct_force_vector`.

In [ ]:
# | export
@patch
def correct_force_vector(
    self: DirectStiffnessMethod,
):
    self.F_f_nodes = self.F_f.copy()
    self.F_f_nodes[self.translationals] = 0

    self.F_f_corrected = self.F_f_calculated - self.F_f_nodes

In [ ]:
System.translationals

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False])

In [ ]:
~System.translationals

array([False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
System.correct_force_vector()

In [ ]:
System.F_f

array([ 100000.,       0., -100000.,       0.,       0.,       0.,
             0.,       0.,       0.,  150000.,       0.,   50000.])

In [ ]:
System.F_f_nodes

array([     0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0., 150000.,      0.,  50000.])

In [ ]:
System.F_f_calculated

array([ 100000.        ,       0.        , -100000.        ,
             0.        ,       0.        ,   65277.77777778,
        -53703.7037037 ,       0.        ,       0.        ,
        -46296.2962963 ,       0.        ,   34722.22222222])

In [ ]:
System.F_f_corrected

array([ 100000.        ,       0.        , -100000.        ,
             0.        ,       0.        ,   65277.77777778,
        -53703.7037037 ,       0.        ,       0.        ,
       -196296.2962963 ,       0.        ,  -15277.77777778])

$$S_e = \frac{E \cdot A}{l^2} \cdot \begin{bmatrix} -l_x & -l_y & -l_z & l_x & l_y & l_z \end{bmatrix} $$


In [ ]:
# | export
@patch
def create_internal_force_matrice(
    self: DirectStiffnessMethod, E, A, x_1, y_1, z_1, x_2, y_2, z_2
):
    l_x, l_y, l_z = self.calculate_differences(x_1, y_1, z_1, x_2, y_2, z_2)

    l = self.calculate_distance(l_x, l_y, l_z)

    S_e = (E * A / (l**2)) * np.array([[-l_x, -l_y, -l_z, l_x, l_y, l_z]])

    return S_e

In [ ]:
# | export
@patch
def create_internal_force_matrice_in_df(self: DirectStiffnessMethod, row):
    E = row["Modulus_of_elasticity"]
    A = row["Cross-sectional_area"]
    x_1 = row["x_1"]
    y_1 = row["y_1"]
    z_1 = row["z_1"]
    x_2 = row["x_2"]
    y_2 = row["y_2"]
    z_2 = row["z_2"]

    S_e = self.create_internal_force_matrice(
        E, A, x_1, y_1, z_1, x_2, y_2, z_2
    )
    return S_e

In [ ]:
# | export
@patch
def create_internal_force_matrices(self: DirectStiffnessMethod):
    self.bars_extended["Internal_force_matrice"] = self.bars_extended.apply(
        self.create_internal_force_matrice_in_df, axis=1
    )

In [ ]:
System.create_internal_force_matrices()

In [ ]:
System.bars_extended["Internal_force_matrice"]

0    [[-0.0, -0.0, 333.3333333333333, 0.0, 0.0, -33...
1              [[250.0, -0.0, -0.0, -250.0, 0.0, 0.0]]
2          [[160.0, -0.0, -120.0, -160.0, 0.0, 120.0]]
Name: Internal_force_matrice, dtype: object

In [ ]:
System.u_f

array([ 214.81481481,    0.        , -195.83333333,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [ ]:
System.bars_extended

,Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity,x_1,y_1,z_1,Degrees_of_freedom_1,x_2,y_2,z_2,Degrees_of_freedom_2,Internal_force_matrice
0,1,2,1,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",0.0,0.0,0.0,"[3, 4, 5]","[[-0.0, -0.0, 333.3333333333333, 0.0, 0.0, -33..."
1,2,2,3,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",-4000.0,0.0,3000.0,"[6, 7, 8]","[[250.0, -0.0, -0.0, -250.0, 0.0, 0.0]]"
2,3,2,4,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",-4000.0,0.0,6000.0,"[9, 10, 11]","[[160.0, -0.0, -120.0, -160.0, 0.0, 120.0]]"


$$N = S_e \cdot u_e $$

In [ ]:
# | export
@patch
def calculate_normal_force_in_df(self: DirectStiffnessMethod, row):
    degrees_of_freedom = np.concatenate(
        (row["Degrees_of_freedom_1"], row["Degrees_of_freedom_2"])
    )
    N = row["Internal_force_matrice"] @ self.u_f[degrees_of_freedom]
    return N[0]

In [ ]:
# | export
@patch
def calculate_normal_force(self: DirectStiffnessMethod):
    self.bars_extended["Normal_force"] = self.bars_extended.apply(
        self.calculate_normal_force_in_df, axis=1
    )

In [ ]:
System.calculate_normal_force()

In [ ]:
System.bars_extended

,Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity,x_1,y_1,z_1,Degrees_of_freedom_1,x_2,y_2,z_2,Degrees_of_freedom_2,Internal_force_matrice,Normal_force
0,1,2,1,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",0.0,0.0,0.0,"[3, 4, 5]","[[-0.0, -0.0, 333.3333333333333, 0.0, 0.0, -33...",-65277.777778
1,2,2,3,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",-4000.0,0.0,3000.0,"[6, 7, 8]","[[250.0, -0.0, -0.0, -250.0, 0.0, 0.0]]",53703.703704
2,3,2,4,1000.0,1000.0,0.0,0.0,3000.0,"[0, 1, 2]",-4000.0,0.0,6000.0,"[9, 10, 11]","[[160.0, -0.0, -120.0, -160.0, 0.0, 120.0]]",57870.370370


In [ ]:
System.bars_extended["Normal_force"]

0   -65277.777778
1    53703.703704
2    57870.370370
Name: Normal_force, dtype: float64

In [ ]:
System.degrees_of_freedom

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [ ]:
System.translationals

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False])

In [ ]:
System.u_f

array([ 214.81481481,    0.        , -195.83333333,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [ ]:
System.degrees_of_freedom[System.translationals]

array([0, 2])

In [ ]:
System.u_f[System.translationals]

array([ 214.81481481, -195.83333333])

In [ ]:
System.u_f[2::3]

array([-195.83333333,    0.        ,    0.        ,    0.        ])

In [ ]:
System.nodes

,Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z,Degrees_of_freedom
0,2,0.0,0.0,3000.0,False,True,False,"[0, 1, 2]"
1,1,0.0,0.0,0.0,True,True,True,"[3, 4, 5]"
2,3,-4000.0,0.0,3000.0,True,True,True,"[6, 7, 8]"
3,4,-4000.0,0.0,6000.0,True,True,True,"[9, 10, 11]"


In [ ]:
# | export
@patch
def create_displacment_df(self: DirectStiffnessMethod):
    self.dimensions_names = ["X", "Y", "Z"]
    for index, name in enumerate(self.dimensions_names):
        self.nodes[f"Displacement_{name}"] = self.u_f[
            index :: self.dimensions
        ]

    self.translational_names = [
        "Translational_X",
        "Translational_X",
        "Translational_X",
    ]
    self.displacment_df = self.nodes[
        ~(self.nodes[self.translational_names].all(axis=1))
    ]

    self.displacment_df = self.displacment_df[
        ["Node", "Displacement_X", "Displacement_Y", "Displacement_Z"]
    ]

In [ ]:
System.create_displacment_df()

In [ ]:
System.displacment_df.style.hide(axis="index")

Node,Displacement_X,Displacement_Y,Displacement_Z
2,214.814815,0.000000,-195.833333


In [ ]:
System.F_f_corrected

array([ 100000.        ,       0.        , -100000.        ,
             0.        ,       0.        ,   65277.77777778,
        -53703.7037037 ,       0.        ,       0.        ,
       -196296.2962963 ,       0.        ,  -15277.77777778])

In [ ]:
# | export
@patch
def create_force_df(self: DirectStiffnessMethod):
    for index, name in enumerate(self.dimensions_names):
        self.nodes[f"Force_{name}"] = self.F_f_corrected[
            index :: self.dimensions
        ]

    self.force_df = self.nodes[
        (self.nodes[self.translational_names].all(axis=1))
    ]

    self.force_df = self.force_df[["Node", "Force_X", "Force_Y", "Force_Z"]]

In [ ]:
System.create_force_df()

In [ ]:
System.force_df.style.hide(axis="index")

Node,Force_X,Force_Y,Force_Z
1,0.000000,0.000000,65277.777778
3,-53703.703704,0.000000,0.000000
4,-196296.296296,0.000000,-15277.777778


In [ ]:
# | export
@patch
def create_normal_force_df(self: DirectStiffnessMethod):
    self.normal_force_df = self.bars_extended[
        ["Bar", "Normal_force"]
    ].copy()
    self.normal_force_df["Type_of_normal_force"] = self.normal_force_df[
        "Normal_force"
    ].apply(
        lambda x: (
            "Tensile force"
            if x > 0
            else ("Compressive force" if x < 0 else "Zero-force")
        )
    )

In [ ]:
System.create_normal_force_df()

In [ ]:
System.normal_force_df.style.hide(axis="index")

Bar,Normal_force,Type_of_normal_force
1,-65277.777778,Compressive force
2,53703.703704,Tensile force
3,57870.370370,Tensile force


In [ ]:
# | hide
nbdev.nbdev_export()